In [1]:
import sys
import pandas as pd

%load_ext autoreload
%autoreload 2|

In [2]:
sys.path.append('../')

In [3]:
from asynctcn.dataset import DassaultDataset
from asynctcn.model import TCN

In [4]:
dataset = DassaultDataset('../data/Dassault_Cache/train/', 500)

In [32]:
dataset[2000000][0]

array([[0.95883311, 0.36848628, 0.5861016 , ..., 0.43298845, 0.02498365,
        0.57980902],
       [0.95572834, 0.15588258, 0.54197029, ..., 0.36714388, 0.04981065,
        0.41651629],
       [0.9091907 , 0.79725448, 0.92282474, ..., 0.82108014, 0.02976096,
        0.61577464]])

In [13]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset,
                        batch_size=1,
                        shuffle=True)

In [6]:
summary_df = pd.read_csv('../data/Dassault_Cache/train/flight1/Summary.csv', sep=';')
input_channels = len(summary_df[(summary_df['ENABLED'] == 1) & (summary_df['TYPE'] == 'INPUT')])
outputs = [(l, 1) for l in summary_df[(summary_df['ENABLED'] == 1) & (summary_df['TYPE'] == 'OUTPUT')]['LABEL'].values]

channel_sizes = [50]*8

model = TCN(input_channels, outputs, channel_sizes, kernel_size=3, dropout=0.2)

In [10]:
import torch
import torch.nn.functional as F

In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

In [22]:
model.train()
batch_idx = 1
total_loss = 0

for i, (X, y, lab) in enumerate(dataloader):
    optimizer.zero_grad()
    output = model(X.float(), int(lab[0]))
    loss = F.mse_loss(output, y.float())
    loss.backward()
#     if args.clip > 0:
#         torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
    optimizer.step()
    total_loss += loss.item()

    if i % 5 == 0:
        cur_loss = total_loss / 5
        processed = min(i+8, 10000)
        print('Train Epoch: {:2d} [{:6d}/{:6d} ({:.0f}%)]\tLearning rate: {:.4f}\tLoss: {:.6f}'.format(
            1, processed, 100000, 100.*processed/100000, 0.005, cur_loss))
        total_loss = 0

/opt/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  


Train Epoch:  1 [     8/100000 (0%)]	Learning rate: 0.0050	Loss: 0.013474
Train Epoch:  1 [    13/100000 (0%)]	Learning rate: 0.0050	Loss: 0.093731
Train Epoch:  1 [    18/100000 (0%)]	Learning rate: 0.0050	Loss: 0.102379
Train Epoch:  1 [    23/100000 (0%)]	Learning rate: 0.0050	Loss: 0.086932
Train Epoch:  1 [    28/100000 (0%)]	Learning rate: 0.0050	Loss: 0.209128
Train Epoch:  1 [    33/100000 (0%)]	Learning rate: 0.0050	Loss: 0.057062
Train Epoch:  1 [    38/100000 (0%)]	Learning rate: 0.0050	Loss: 0.142867
Train Epoch:  1 [    43/100000 (0%)]	Learning rate: 0.0050	Loss: 0.084415
Train Epoch:  1 [    48/100000 (0%)]	Learning rate: 0.0050	Loss: 0.057352
Train Epoch:  1 [    53/100000 (0%)]	Learning rate: 0.0050	Loss: 0.190942
Train Epoch:  1 [    58/100000 (0%)]	Learning rate: 0.0050	Loss: 0.111292
Train Epoch:  1 [    63/100000 (0%)]	Learning rate: 0.0050	Loss: 0.083019
Train Epoch:  1 [    68/100000 (0%)]	Learning rate: 0.0050	Loss: 0.088981
Train Epoch:  1 [    73/100000 (0%)]	L

Train Epoch:  1 [   568/100000 (1%)]	Learning rate: 0.0050	Loss: 0.081977
Train Epoch:  1 [   573/100000 (1%)]	Learning rate: 0.0050	Loss: 0.099779
Train Epoch:  1 [   578/100000 (1%)]	Learning rate: 0.0050	Loss: 0.114555
Train Epoch:  1 [   583/100000 (1%)]	Learning rate: 0.0050	Loss: 0.076340


KeyboardInterrupt: 